In [1]:
import pandas as pd
import numpy as np
import os
import seaborn

os.chdir('C:/Users/최민석/OneDrive - UOS/바탕 화면/대학생활/대외활동/Kaggle/playground-series-s3e20')
train=pd.read_csv('train.csv')
train.head()


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'C:/Users/최민석/OneDrive - UOS/바탕 화면/대학생활/대외활동/Kaggle/playground-series-s3e20'

In [ ]:
train.info()